# Evaluation Multiscale Interpolation (MIOST): 

This notebook presents the evaluation of the SSH reconstructions based on the Dynamic Interpolation method ([Ubelmann et al., 2021](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/2020JC016560)) and performed for the **"2021a_SSH_mapping_OSE" ocean data challenge**. 

In [ ]:
import os
import sys
sys.path.append('..')
import logging
import pandas as pd

In [ ]:
from src.mod_inout import *
from src.mod_interp import *
from src.mod_stats import *
from src.mod_spectral import *
from src.mod_plot import *

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

### Study Area & Ouput Parameters

In [ ]:
# study area
lon_min = 295.
lon_max = 305.
lat_min = 33.
lat_max = 43.
is_circle = False
time_min = '2017-01-01'
time_max = '2017-12-31'

# Outputs
bin_lat_step = 1.
bin_lon_step = 1.
bin_time_step = '1D'
output_directory = '../results'
os.mkdir(output_directory)
output_filename = f'{output_directory}/stat_OSE_MIOST_{time_min}_{time_max}_{lon_min}_{lon_max}_{lat_min}_{lat_max}.nc'
output_filename_timeseries = f'{output_directory}/stat_timeseries_OSE_MIOST_{time_min}_{time_max}_{lon_min}_{lon_max}_{lat_min}_{lat_max}.nc'

# Spectral parameter
# C2 parameter
delta_t = 0.9434  # s
velocity = 6.77   # km/s
delta_x = velocity * delta_t
lenght_scale = 1000 # km
output_filename_spectrum = f'{output_directory}/psd_OSE_MIOST_{time_min}_{time_max}_{lon_min}_{lon_max}_{lat_min}_{lat_max}.nc'

### Open your AVISO+ session: fill the ```<AVISO_LOGIN>``` and ```<AVISO_PWD>``` items below

In [ ]:
my_aviso_session = rq.Session()
my_aviso_session.auth = ("<AVISO_LOGIN>", "<AVISO_PWD>")
url_alongtrack = 'https://tds.aviso.altimetry.fr/thredds/dodsC/2021a-SSH-mapping-OSE-along-track-data'
url_map = 'https://tds.aviso.altimetry.fr/thredds/dodsC/2021a-SSH-mapping-OSE-grid-data'

### Read L3 datasets

In [ ]:
# independent along-track
alontrack_independent_dataset = f'{url_alongtrack}/dt_gulfstream_c2_phy_l3_20161201-20180131_285-315_23-53.nc'
# Read along-track
ds_alongtrack = read_l3_dataset_from_aviso(alontrack_independent_dataset, 
                                           my_aviso_session,
                                           lon_min=lon_min, 
                                           lon_max=lon_max, 
                                           lat_min=lat_min, 
                                           lat_max=lat_max, 
                                           time_min=time_min, 
                                           time_max=time_max)
ds_alongtrack

### Read L4 dataset and interpolate onto along-track positions

In [ ]:
# series of maps to evaluate
gridded_dataset = [f'{url_map}/OSE_ssh_mapping_MIOST.nc', my_aviso_session] 
# Interpolate maps onto alongtrack dataset
time_alongtrack, lat_alongtrack, lon_alongtrack, ssh_alongtrack, ssh_map_interp = interp_on_alongtrack(gridded_dataset, 
                                                                                                       ds_alongtrack,
                                                                                                       lon_min=lon_min, 
                                                                                                       lon_max=lon_max, 
                                                                                                       lat_min=lat_min, 
                                                                                                       lat_max=lat_max, 
                                                                                                       time_min=time_min, 
                                                                                                       time_max=time_max,
                                                                                                       is_circle=is_circle)

### Compute statistical score

In [ ]:
leaderboard_nrmse, leaderboard_nrmse_std = compute_stats(time_alongtrack, 
                                                         lat_alongtrack, 
                                                         lon_alongtrack, 
                                                         ssh_alongtrack, 
                                                         ssh_map_interp, 
                                                         bin_lon_step,
                                                         bin_lat_step, 
                                                         bin_time_step,
                                                         output_filename,
                                                         output_filename_timeseries)

In [ ]:
plot_spatial_statistics(output_filename)

In [ ]:
plot_temporal_statistics(output_filename_timeseries)

### Compute spectral scores

In [ ]:
compute_spectral_scores(time_alongtrack, 
                        lat_alongtrack, 
                        lon_alongtrack, 
                        ssh_alongtrack, 
                        ssh_map_interp, 
                        lenght_scale,
                        delta_x,
                        delta_t,
                        output_filename_spectrum)

In [ ]:
leaderboard_psds_score = plot_psd_score(output_filename_spectrum)

In [ ]:
# Print leaderboard
data = [['MIOST', 
         leaderboard_nrmse, 
         leaderboard_nrmse_std, 
         int(leaderboard_psds_score),
        'Multiscale mapping',
        'example_eval_miost.ipynb']]
Leaderboard = pd.DataFrame(data, 
                           columns=['Method', 
                                    "µ(RMSE) ", 
                                    "σ(RMSE)", 
                                    'λx (km)',  
                                    'Notes',
                                    'Reference'])
print("Summary of the leaderboard metrics:")
Leaderboard
print(Leaderboard.to_markdown())